In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras_tuner import RandomSearch
import keras_tuner as kt

# Sample data
input1 = np.random.rand(1000, 8)
input2 = np.random.rand(1000, 8)
output1 = np.random.rand(1000, 1)
output2 = np.random.rand(1000, 1)

# Model definition with Normalization layers
def build_model(hp):
    input_a = keras.layers.Input(shape=(8,), name='input_a')
    normalized_a = keras.layers.Normalization()(input_a)

    input_b = keras.layers.Input(shape=(8,), name='input_b')
    normalized_b = keras.layers.Normalization()(input_b)

    x1 = keras.layers.Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=32), activation='relu')(normalized_a)
    x2 = keras.layers.Dense(units=hp.Int('dense_2_units', min_value=32, max_value=128, step=32), activation='relu')(normalized_b)
    
    merged = keras.layers.concatenate([x1, x2])
    out_a = keras.layers.Dense(1, name='out_a')(merged)
    out_b = keras.layers.Dense(1, name='out_b')(merged)

    model = keras.models.Model(inputs=[input_a, input_b], outputs=[out_a, out_b])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss={'out_a': 'mse', 'out_b': 'mse'},
                  metrics={'out_a': 'mae', 'out_b': 'mae'})
    return model

# Keras Tuner setup
tuner = RandomSearch(
    build_model,
    objective=kt.Objective("val_out_a_mae", direction="min"),
    overwrite=True,
    max_trials=5,
    executions_per_trial=3,
    directory='output_dir',
    project_name='keras_tuner_example'
)

# Search for the best hyperparameters
tuner.search([input1, input2], [output1, output2], epochs=5, validation_split=0.2)

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Adapt the normalization layers in the best model
normalization_layer_a = best_model.get_layer(name='normalization')  # Note: Keras automatically appends an index to the layer name
normalization_layer_a.adapt(input1)

normalization_layer_b = best_model.get_layer(name='normalization_1')
normalization_layer_b.adapt(input2)

# Fit the best model to the data
history = best_model.fit([input1, input2], [output1, output2], epochs=10, validation_split=0.2)


Trial 5 Complete [00h 00m 08s]
val_out_a_mae: 0.2617969314257304

Best val_out_a_mae So Far: 0.25833870967229206
Total elapsed time: 00h 00m 40s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
25/25 [==============================] - 2s 16ms/step - loss: 0.2681 - out_a_loss: 0.1530 - out_b_loss: 0.1151 - out_a_mae: 0.3176 - out_b_mae: 0.2777 - val_loss: 0.2143 - val_out_a_loss: 0.1031 - val_out_b_loss: 0.1112 - val_out_a_mae: 0.2675 - val_out_b_mae: 0.2813
Epoch 2/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1843 - out_a_loss: 0.0893 - out_b_loss: 0.0950 - out_a_mae: 0.2476 - out_b_mae: 0.2557 - val_loss: 0.1912 - val_out_a_loss: 0.0958 - val_out_b_loss: 0.0955 - val_out_a_mae: 0.2582 - val_out_b_mae: 0.2606
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1715 - out_a_loss: 0.0825 - out_b_loss: 0.0890 - out_a_mae: 0.2418 - out_b_mae: 0.2505 - val_loss: 0.1922 - val_out_a_loss: 0.0928 - val_out_b_loss: 0.0994 - val_out_a_mae: 0.2606 - val_

In [2]:
from keras.datasets import fashion_mnist

In [3]:
(x_train_full,y_train_full),(x_test,y_test) = fashion_mnist.load_data()

In [4]:
x_train,x_val = x_train_full[5000:]/255,x_train_full[:5000]/255

In [5]:
y_train,y_val = y_train_full[5000:],y_train_full[:5000]

In [6]:
def build_mod(hp):
    input_ = keras.layers.Input(shape=[28,28])
    flatten= keras.layers.Flatten()(input_)
    hidden1=keras.layers.Dense(units=hp.Int('dense_1',min_value=30,max_value=300,step=30),activation='relu')(flatten)
    hidden2=keras.layers.Dense(units=hp.Int('dense_2',min_value=30,max_value=300,step=30),activation='relu')(hidden1)
    output = keras.layers.Dense(10,activation='softmax')(hidden2)
    optimizer = hp.Choice('optimizer',values=['sgd','adam'])
  
    if optimizer =='sgd':
        optimizer = keras.optimizers.SGD(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    else:
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model = keras.Model(inputs=[input_],outputs=[output])
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer = optimizer,
                 metrics=['accuracy'])
    return model
    
    
    

In [7]:
tuner = kt.RandomSearch(build_mod,
                     objective='val_accuracy',
                     overwrite=True,
                     max_trials=5,
                     executions_per_trial=3,
                     directory='output_diry',
                     project_name='keras_tuner_example_1',
                    )

In [ ]:
tuner.search(x_train,y_train,epochs=10,validation_data=(x_val,y_val))

Trial 4 Complete [00h 03m 20s]
val_accuracy: 0.8891333142916361

Best val_accuracy So Far: 0.8891333142916361
Total elapsed time: 00h 11m 30s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
120               |300               |dense_1
270               |90                |dense_2
sgd               |adam              |optimizer
0.01              |0.0001            |learning_rate

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7829 - accuracy: 0.7417 - val_loss: 0.5440 - val_accuracy: 0.8178
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5075 - accuracy: 0.8244 - val_loss: 0.4935 - val_accuracy: 0.8304
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4599 - accuracy: 0.8386 - val_loss: 0.4377 - val_accuracy: 0.8472
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4323 - accuracy: 0.8480 - val_loss: 0.4347 - val_accuracy: 0.8476
Epoch 5/1

In [ ]:
top3_params = tuner.get_best_hyperparameters(num_trials=3)


In [ ]:
top3_params[0].values

In [ ]:
top3_model = tuner.get_best_models(num_models=3)
best_model = top3_model[0]


In [ ]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

In [ ]:
h = best_model.fit(x_train,y_train,epochs=10,validation_data=(x_val,y_val))

In [ ]:
import pandas as pd
dd = pd.DataFrame(h.history)

In [ ]:
dd


In [ ]:
best_model.evaluate(x_test,y_test)